In [1]:
import requests

ogc_api_endpoint = "http://zoo-project-dru-service/alice/ogc-api/processes"

r = requests.get(ogc_api_endpoint)

print(r.json())


{'processes': [{'id': 'echo', 'title': 'Echo input', 'description': 'Simply echo the value provided as input', 'mutable': False, 'version': '2.0.0', 'metadata': [{'title': 'Demo'}], 'jobControlOptions': ['sync-execute', 'async-execute', 'dismiss'], 'outputTransmission': ['value', 'reference'], 'links': [{'rel': 'self', 'type': 'application/json', 'title': 'Process Description', 'href': 'http://localhost:8080/alice/ogc-api/processes/echo'}]}], 'links': [{'rel': 'self', 'type': 'application/json', 'href': 'http://localhost:8080/alice/ogc-api/processes'}], 'numberTotal': 1}


In [2]:
headers = {}
response = requests.post(
    f"{ogc_api_endpoint}",
    headers = dict([("Content-Type", "application/json")], **headers),
    json = {
      "executionUnit": {
        "href": "https://github.com/Terradue/ogc-eo-application-package-hands-on/releases/download/1.5.0/app-water-bodies-cloud-native.1.5.0.cwl",
        "type": "application/cwl"
      }
    }
)

In [3]:
print(response.json())

{'id': 'water_bodies', 'title': 'Water bodies detection based on NDWI and otsu threshold', 'description': 'Water bodies detection based on NDWI and otsu threshold', 'mutable': True, 'version': '1.4.1', 'metadata': [{'role': 'https://schema.org/softwareVersion', 'value': '1.4.1'}], 'outputTransmission': ['value', 'reference'], 'jobControlOptions': ['async-execute', 'dismiss'], 'links': [{'rel': 'http://www.opengis.net/def/rel/ogc/1.0/execute', 'type': 'application/json', 'title': 'Execute End Point', 'href': 'http://localhost:8080/alice/ogc-api/processes/water_bodies/execution'}]}


In [4]:
print(response.status_code)

201


In [5]:
print(response.text)

{"id":"water_bodies","title":"Water bodies detection based on NDWI and otsu threshold","description":"Water bodies detection based on NDWI and otsu threshold","mutable":true,"version":"1.4.1","metadata":[{"role":"https://schema.org/softwareVersion","value":"1.4.1"}],"outputTransmission":["value","reference"],"jobControlOptions":["async-execute","dismiss"],"links":[{"rel":"http://www.opengis.net/def/rel/ogc/1.0/execute","type":"application/json","title":"Execute End Point","href":"http://localhost:8080/alice/ogc-api/processes/water_bodies/execution"}]}



In [6]:
headers = {
    'accept': '*/*',
    'Prefer': 'respond-async;return=representation',
    'Content-Type': 'application/json'
}
data = {
    "inputs": {
        "stac_items": [
            "https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items/S2B_10TFK_20210713_0_L2A",
            "https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items/S2A_10TFK_20220524_0_L2A"
        ],
        "aoi": "-121.399,39.834,-120.74,40.472",
        "epsg": "EPSG:4326",
        "bands": [
            "green",
            "nir"
        ]
    }
}

response = requests.post(f"{ogc_api_endpoint}/water_bodies/execution", headers=headers, json=data)

print(response.status_code)
print(response.json())

201
{'jobID': '416d9012-8b78-11ef-be1a-52ded859c017', 'type': 'process', 'processID': 'water_bodies', 'created': '2024-10-16T04:36:48.586Z', 'started': '2024-10-16T04:36:48.586Z', 'updated': '2024-10-16T04:36:48.586Z', 'status': 'running', 'message': 'ZOO-Kernel accepted to run your service!', 'links': [{'title': 'Status location', 'rel': 'monitor', 'type': 'application/json', 'href': 'http://localhost:8080/alice/ogc-api/jobs/416d9012-8b78-11ef-be1a-52ded859c017'}]}


In [7]:
jobid = response.json().get("jobID")

In [8]:
import time
job_url = 'http://zoo-project-dru-service/alice' + '/ogc-api/jobs/' + jobid

headers = {
    'accept': 'application/json',
}

while True:
    response = requests.get(job_url, headers=headers)

    if response.status_code == 200:
        data = response.json()
        status = data.get("status")
        if status == "successful":
            print("Job status:", status)
            break  # Exit the loop if OK
        else:
            print("Job status:", status)
            time.sleep(10)
    else:
        print("Error:", response.status_code)
        break  

Job status: running
Job status: running
Job status: running
Job status: running
Job status: running
Job status: running
Job status: running
Job status: running
Job status: running
Job status: running
Job status: running
Job status: running
Job status: running
Job status: running
Job status: running
Job status: running
Job status: running
Job status: running
Job status: running
Job status: running
Job status: successful


In [9]:
job_results = 'http://zoo-project-dru-service/alice' + '/ogc-api/jobs/' + jobid + "/results"

headers = {
    'accept': 'application/json',
}
response = requests.get(job_url, headers=headers)

print(response.json())

{'jobID': '416d9012-8b78-11ef-be1a-52ded859c017', 'type': 'process', 'processID': 'water_bodies', 'created': '2024-10-16T04:36:48.586Z', 'started': '2024-10-16T04:36:48.586Z', 'finished': '2024-10-16T04:40:21.469Z', 'updated': '2024-10-16T04:40:21.204Z', 'status': 'successful', 'message': 'ZOO-Kernel successfully run your service!', 'links': [{'title': 'Status location', 'rel': 'monitor', 'type': 'application/json', 'href': 'http://localhost:8080/alice/ogc-api/jobs/416d9012-8b78-11ef-be1a-52ded859c017'}, {'title': 'Result location', 'rel': 'http://www.opengis.net/def/rel/ogc/1.0/results', 'type': 'application/json', 'href': 'http://localhost:8080/alice/ogc-api/jobs/416d9012-8b78-11ef-be1a-52ded859c017/results'}, {'href': 'http://localhost:8080/alice/temp/water_bodies-416d9012-8b78-11ef-be1a-52ded859c017/node_crop.log', 'title': 'Tool log node_crop.log', 'rel': 'related', 'type': 'text/plain'}, {'href': 'http://localhost:8080/alice/temp/water_bodies-416d9012-8b78-11ef-be1a-52ded859c017/

In [10]:
response.json()

{'jobID': '416d9012-8b78-11ef-be1a-52ded859c017',
 'type': 'process',
 'processID': 'water_bodies',
 'created': '2024-10-16T04:36:48.586Z',
 'started': '2024-10-16T04:36:48.586Z',
 'finished': '2024-10-16T04:40:21.469Z',
 'updated': '2024-10-16T04:40:21.204Z',
 'status': 'successful',
 'message': 'ZOO-Kernel successfully run your service!',
 'links': [{'title': 'Status location',
   'rel': 'monitor',
   'type': 'application/json',
   'href': 'http://localhost:8080/alice/ogc-api/jobs/416d9012-8b78-11ef-be1a-52ded859c017'},
  {'title': 'Result location',
   'rel': 'http://www.opengis.net/def/rel/ogc/1.0/results',
   'type': 'application/json',
   'href': 'http://localhost:8080/alice/ogc-api/jobs/416d9012-8b78-11ef-be1a-52ded859c017/results'},
  {'href': 'http://localhost:8080/alice/temp/water_bodies-416d9012-8b78-11ef-be1a-52ded859c017/node_crop.log',
   'title': 'Tool log node_crop.log',
   'rel': 'related',
   'type': 'text/plain'},
  {'href': 'http://localhost:8080/alice/temp/water_bo

In [12]:
ogc_api_endpoint = "http://zoo-project-dru-service/alice/ogc-api/processes"

r = requests.get(f"{ogc_api_endpoint}/water_bodies")

r.json()

{'id': 'water_bodies',
 'title': 'Water bodies detection based on NDWI and otsu threshold',
 'description': 'Water bodies detection based on NDWI and otsu threshold',
 'mutable': True,
 'version': '1.4.1',
 'metadata': [{'role': 'https://schema.org/softwareVersion',
   'value': '1.4.1'}],
 'outputTransmission': ['value', 'reference'],
 'jobControlOptions': ['async-execute', 'dismiss'],
 'links': [{'rel': 'http://www.opengis.net/def/rel/ogc/1.0/execute',
   'type': 'application/json',
   'title': 'Execute End Point',
   'href': 'http://localhost:8080/alice/ogc-api/processes/water_bodies/execution'}],
 'inputs': {'aoi': {'title': 'area of interest',
   'description': 'area of interest as a bounding box',
   'schema': {'type': 'string'}},
  'bands': {'title': 'bands used for the NDWI',
   'description': 'bands used for the NDWI',
   'schema': {'type': 'string', 'default': "['green', 'nir']"}},
  'epsg': {'title': 'EPSG code',
   'description': 'EPSG code',
   'schema': {'type': 'string', 